## Time Series and Autocorrelations

In this video we'll be talking about time series. And, as the name implies, these are simply a series of
data points that are ordered in time. We will first plot a time series and then discuss some features of
this graph such as periodicity and lag. After that, we'll use the data to create an autocorrelation plot,
which is a helpful tool for checking whether or not we have randomness in the data, or if patterns and
correlations actually exist.

![time series](assets/timeseries.png)

As the image above shows, a time series plot is similar to other graphs you may have already encountered,
only that each point along the x-axis represents a point in time. So for instance, the vertical line might
represent the actual and 5-day forecasted temperature for August 1, 2017.

In [ ]:
#Throughout this lecture, I'll be using a historical dataset that logs weekly case reports 
#of the measles disease from 1948-1967 in England and Wales. Let's go ahead and read in 
#this file and take a look at the data.

# Let's bring in pandas and numpy
import pandas as pd
import numpy as np

# And let's read in the dataset
df = pd.read_table('assets/ewcitmeas.txt', delim_whitespace=True)
df.head(10)

In [ ]:
# There are several proprocessing steps we should conduct before moving on to more detailed analyses. For
# instance, there are asterisks where we are missing values, and the date, month, and year format isn't
# particularly easy to work with. Let's address this by renaming the date-related column names, so that they
# are compatible with Panda's to_datetime function, which will automatically generate a single date-time
# column, in date-time data types, which are more convenient to work with. The documention can be found here:
# https://pandas.pydata.org/pandas-docs/stable/user_guide/timeseries.html
df.rename(columns={'DD': 'day', 'MM': 'month', 'YY': 'year'}, inplace=True)
# Let's change the year to four digit format
df['year'] = df['year'] + 1900
# And now let's turn this into an actual datetime field
datetime = pd.to_datetime(df[['year', 'month', 'day']])
df['Datetime'] = datetime
df.head()

In [ ]:
# Pandas has excellent time series data features as it was originally created by Wes McKinney to deal with
# financial data. I'll be honest, I don't use time series analyses a lot in my work, so I often find I have to
# reference the pandas documentation, but when I do I'm constantly amazed at some of the things you can do
# right out of the box.

# Let's replace asterisks with 'nan' values, and then use the dropna function to remove these from our
# dataframe. Also, if you try to immediately make a plot, you'll notice some weird behavior, which is due to
# the fact that the data types of the other columns are actually all strings right now. So, for now, since
# we'll only be looking at the cases in London, we'll cast that column to an int.
df = df.replace('*', np.nan)
df = df.dropna()
df['London'] = df['London'].astype('int64')
df.head()

In [ ]:
df["Datetime"]

In [ ]:
# Ok, data cleaning, it's in every project! :)

# Let's bring in pyplot and setup matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# One little trick I'll show you here is that you can set the default figure size using the rcParams
# settings. This is a global matplotlib dictionary of default values, and I often find I set a few things
# at the top of my notebooks then I can override them if I need to throughout. Here we'll set the figure
# size, but you can change default fonts, colors, etc. Note that this must be done after %matplotlib inline
import matplotlib as mpl
mpl.rcParams['figure.figsize'] = [16.0,4.0]

# Now we're ready to plot the time series, let's look at values for London over time
plt.plot(df['Datetime'], df['London'])

### Periodity and Autocorrelations

You may have noticed that there seem to be some spikes that happen every so often in the previous plot, and
so one of the questions one might be interested in is whether or not there is some pattern to these
increases in reporting. Or in other words, is the time series *periodic* in nature, similar to how the four
seasons change in a generally predictable pattern year after year (with some exceptions depending on where
you live, of course).

Our investigation in this lecture is to comparing an existing time series of data with a time-lagged version
of itself (hence the prefix "auto-", which means self). If they are correlated, then we'd see a positive
value (negative if they are anti-correlated). Otherwise, if there's no relationship, we'd expect to see a
value close to zero. 

We can measure this correlation by "sliding" the second function over the first function, and measuring the
amount of overlap, like so:

![convolution](assets/convolution.gif)

If you would like to read more about the details behind how this method works, I highly recommend the
wikipedia article on the topic of convolution: https://en.wikipedia.org/wiki/Convolution

But, lets do a simple example of our own...

In [ ]:
# To help solidify the concept of autocorrelation a bit, let's do a bit of autocorrelation by hand. 
# The first thing we want to do is center our data by subtracting off the mean.
x = df['London'] - df['London'].mean()

# Now we want to see the correlation between two functions (represented as arrays of values). Since this
# is autocorrelation, we're comparing the data in x against itself but we want to shift the comparison
# data. We can do this using the mode='full' parameter of numpy's correlate function. The details behind
# this are outside the scope of this lecture, but if you would like to learn more about signal comparison
# I recommend checking out the scipy.signal docs here:
# https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.correlate.html
autocorr = np.correlate(x, x, mode='full')

# The result of this correlation is a new ndarray which is twice the size of x. This new ndarray is
# symmetric in that the first half and the second half are mirror images of one another. So for our
# analysis we can just look at the second half of this ndarray (which corresponds to the time of lag=0)
autocorr = autocorr[x.size:]

# Finally, we'll normalize the values between 0 and 1 by dividing by the maximum value, this just makes
# things more readable.
autocorr /= autocorr.max()

# Now we can plot 
plt.plot(autocorr)
# And we'll add a reference line at 0
plt.axhline()
# And let's toss in some labels
plt.xlabel("Lag")
plt.ylabel("Normalized correlation")

### Inference and Interpretation

Now that we have an autocorrelation plot, how do we interpret this figure? Well, remember this is a plot of
our signal, that is the function (in a mathematics sense) represented by the data from our original figure,
where we compare that signal to itself, shifting over time. So there is extremely high correlation at the
beginning of the time, since the signals are identical, then as we shift this rapidly drops off as they
become dissimilar, and then increases again as both signals spike. By the end of the time frame there is
essentially no correlation between the signals.

So, what's a reasonable amount of lag? That really depends on your question and the data you are looking at.
Let's dig into this a bit more now that we have some idea of how an autocorrelation plot is generated.

In [ ]:
# I'm going to pivot a bit and just start using the autocorrelation function build into matplotlib. Yup, we
# did all of that as a learning exercise, but the library has the functionality already there for us. To use
# it we just generate a new axes and figure and then call the acorr() function
plt.gca().acorr(x, usevlines=True, maxlags=104, normed=True, lw=2);

# Now, acorr() has a couple of parameters I've set here. First, we pass in our data we want to plot. Next, I
# set the maxlags value to 104. Since our data is weekly, I want to just zoom in and look at the
# autocorrelation for the first two years of data. I also want the data to be normalized, and I set the
# usevlines parameter to True. This changes the plot from a scatter graph to a bar plot (or stem plot) which
# makes it easier to see the area under the correlation. And lastely, I've put a semicolon at the end of the
# line.

# What's up with the semicolon? This is actually not python at all, but it's a Jupyter notebook feature. By
# default the jupyter notebooks display the last variable in a given cell. That's why when we do something
# like df.head() we get a rendered dataframe, because implicitly the jupyter system is changing this to
# display(df.head()). This is normally super handy, but in this case, acorr() not only plots our data, but it
# returns four different variables: lags, which is a vector of the lag periods, c, which is the
# autocorrelation vector, line, which is a number of lines to be plotted, and v, which is the horizontal line
# to be plotted. This turns out to be a *lot* of data, so be default Jupyter will send it all to the display.
# The semicolon is a handly trick that tells Jupyter to change this default behavior and not display the value
# of the last line of the notebook. But our plot should still come out because matplotlib will render it.

In [ ]:
# Right, remember, the autocorrelation is mirrored. Let's set out xaxis values to just look at the time
# lags we are interested in. Now we don't even need to use that semicolon :)
plt.gca().acorr(x, usevlines=True, maxlags=104, normed=True, lw=2)
plt.gca().grid(True)
plt.gca().set_xlim(0,104)

Ok, so, notice that for the first 4 lags, the correlations remain fairly high, above 0.8. This means that
measles cases reported for a given week are very similar in number to those in the subsequent 4 weeks (about
a month). However, if we look half way through the graph, say around week 52, the number of cases is quite
dissimilar. This suggests that there isn't an annual trend in measels cases, that January's number of cases
this year will not reflect next January's number of cases. But taking a bigger picture perspective and
viewing what's happening at an even larger lag shows that week 104 seems to peak again, which gives evidence
that the number of measles cases seems to have a biennial period (or that the data are strongly correlated
once every two years). 

I wanted to come up with another example, because I think autocorrelations are interesting yet can be a bit
complex. So I decided to engage in a little experiment and will share that with you here. While I've been
preparing some of this lecture I've been capturing my various system statistics such as CPU usage and memory
usage to a file. A lot of operating system work is actually based around anticipating or predicting what a
user will do, and then trying to intelligently cache data or computation to make it fast. Traditional hard
disks are a great example of this, where it is expensive to read a small amount of data, but if you know the
user is going to want to read a lot of data you can put it on the disk contiguously to make good use of that
space. So, my question was, could we use autocorrelation plots to explore my system usage?

In [ ]:
# Let's take a look at my data, which I captured with the command dstat -tcm --output dstat_siads521.csv
df=pd.read_csv("assets/dstat_siads521.csv",skiprows=6)
df.head()

In [ ]:
# Ok, our first column is a timestamp, then the next five are CPU statistics, and the last four are memory
# statistics. Let's plot some CPU data
df=df.set_index(df["time"].apply(lambda x: pd.to_datetime(x[6:])))
# Now let's take a minute to look at this data
plt.plot(df["time"],df["idl"])

In [ ]:
# So, by eye, I can't see any meaningful time series related trends in that data. There's certainly some
# interesting bits, like when I start spotify towards the beginning of my data collection and we see a dip
# in CPU availability.

# Now let's look over five minutes of data at the idle time of my CPU and show that as an autocorrelation
plt.gca().acorr(df['idl'], usevlines=True, maxlags=60*5, normed=True, lw=2)
plt.gca().grid(True)
plt.gca().set_xlim(0,60*5)

So this is pretty interesting, and very different from our previous findings. This decreasing slope suggests
that the further you look (the greater you lag) the less the correlation is. So CPU availability is really
well predicticted by the most recent CPU availability, but that doesn't describe well what might happen in 5
minutes. Whereas with the measles case we found that measels in London was well predictive by what happened
in London two years ago, versus one year ago.

In [ ]:
# Ok, one more example. Here's some data I captured of my running activity. Specifically, there is a power
# level in watts that I captured from my power meter and my heart rate in beats per minute. Let's take a
# look
df=pd.read_csv("assets/stryd.csv")
df.head()

In [ ]:
# Let's create out datetime index and do a bit of exploratory data analysis
df=df.set_index(df["timestamp"])

# Let's plot Power, remember we can set label for a legend and format what we want the line to look like
plt.plot(df["timestamp"], df["Power"],'r-', label="Power")
# Now we want to put both of these on the same plot, but since they have different scales (power goes over
# 200, I sure hope heart_date doesn't!) we want to lock the Axes x-axis but let the Y axis for both float
# matplotlib has a handy function for this called twinx()
plt.gca().twinx()
# Now we can plot heard rate
plt.plot(df["timestamp"],df["heart_rate"],'b-', label="Heart Rate")

# Now let's put a little labeling on the plot
plt.gcf().legend(loc="lower center")
plt.gcf().axes[0].set_ylabel("Heart Rate")
plt.gcf().axes[1].set_ylabel("Power")
plt.title("Heart Rate and Power versus Time")

In [ ]:
# Ok, so we see that there is some different ramps up for these two data points. While power can be achieved
# quickly and seems to flux within a small range, heart rate takes awhile to increase then sort of hits a
# maximum. So how would these differ as autocorrelation plots?

# Let's create a new figure 
fig, axs=plt.subplots(2,1,sharex=True,sharey=True,figsize=(12,8))

# Now, I'm going to use matplotlib's autocorrelation function, but I'm going to plot the results myself And,
# this time around, I'm going to use 15 as my number of observations to look for lag over
power_lags,power_auto,*_=plt.acorr(df['Power'],normed=True,maxlags=15)
heart_lags,heart_auto,*_=plt.acorr(df['heart_rate'],normed=True,maxlags=15)

# Wait, what is this *_ all about!? So, I'm trying to layer in here a few more advanced python features. When
# you do tuple unpacking, that is assigning multiple variables to elements in a list like acorr() returns, you
# might not want all of the elements. In this case, I only care about the first two variables, and don't care
# about the lines and such that acorr() went and created. So I can repack all of these up (using the *) and
# assign them to a new variable. In this case, I used the _ as a variable. And that just looks weird, and
# that's the point - it's a convention for a junk variable in python. But actually, it's just a variable, so
# you could actually go get the lines that were created (feel free to print(_) in the next cell to try it!)

# Back to the task at hand. Since we used matplotlib and pyplot, a new figure has been opened. Lets clear it
# here so we can do our own thing
plt.clf()

# Now let's plot our two comparisons as line graphs, since this is mirrored, we start in the middle of each
# ndarray
i=round(len(power_lags)/2)
plt.plot(power_lags[i:],power_auto[i:],'r-',label="Power")
plt.plot(heart_lags[i:],heart_auto[i:],'b-',label="Heart Rate")

# Now let's put a little labeling on the plot
plt.gca().legend(loc="lower center")
plt.gca().set_ylabel("Scaled Autocorrelation")
plt.gca().set_xlabel("Lag")
plt.title("Heart Rate and Power Autocorrelations")

So we can see that the autocorrelation for both of these curves follows a similar linearly decreasing trend
that we observed in our previous example, but that the power actually has less autocorrelation than the
heart rate moment to moment (e.g. over small lag periods). And if you think about this, this seems pretty
reasonable physiologically. It's very hard for me to change my heart rate from one moment to the next
quickly - my body manages my heart rate for me and scales it up and down as needed. But for power, I can
just stop, sending the power to zero, sprint, sending the power high. However, for both of these, there is a
norm to a sort of steady state while I'm running. But I wonder, what if I engaged in different running
strategies, would that change how the autocorrelation plots look? For instance, when you engage in power
training, you aim to keep your power at a certain level, say 300 watts, and to let other variables change
depending upon the training conditions. It might be interesting to compare speed, elevation, heart rate, and
power together, to explore this relationship further.

In this video, I've only scratched the surface when it comes to exploring time series data. For instance,
other tools one might use include partial autocorrelation plots and autoregressive integrated moving average
(ARIMA) models used for forecasting. This is certainly not the last time you'll see time series data,
especially as we become more adept and collecting time series information from our increasingly sensor
augmented world.